In [ ]:
pip install tfds

In [ ]:
pip install tensorflow-model-optimization

In [ ]:
pip install tensorflow-model-optimization tensorflow-probability

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    ResNet50,
    EfficientNetB0,
    resnet50,
    efficientnet
)
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the tf_flowers Dataset
# ---------------------------------------------------
# Create your own train/test split: 80% / 20%
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in tf_flowers
num_classes = info.features['label'].num_classes
nums = num_classes  # keep both variables as you did before

print(f"Number of classes: {num_classes}")

# ---------------------------------------------------
# 2. Convert to NumPy arrays (Optionally Resize)
# ---------------------------------------------------
IMG_SIZE = 128

x_train_list = []
y_train_list = []

for img, label in dataset_train:
    # Resize to reduce memory usage
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

x_test_list = []
y_test_list = []

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

# Convert lists to NumPy arrays
x_train = np.stack(x_train_list).astype("float32")
y_train = np.array(y_train_list)
x_test  = np.stack(x_test_list).astype("float32")
y_test  = np.array(y_test_list)

# ---------------------------------------------------
# 3. Normalize & One-Hot Encode Labels
# ---------------------------------------------------
# Normalize pixel values to [0, 1]
x_train /= 255.0
x_test  /= 255.0

# Convert integer labels to one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test  shape: {x_test.shape}")
print(f"y_test  shape: {y_test.shape}")
print(f"num_classes: {num_classes}, nums: {nums}")


Number of classes: 5
x_train shape: (2936, 128, 128, 3)
y_train shape: (2936, 5)
x_test  shape: (734, 128, 128, 3)
y_test  shape: (734, 5)
num_classes: 5, nums: 5


In [8]:

# We'll define two separate pipelines:
#   - One for ResNet50
#   - One for EfficientNetB0

# ------------------------------
# 2. tf.data Pipeline for ResNet
# ------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    # Resize and apply ResNet-specific preprocessing
    image = tf.image.resize(image, (224, 224))
    image = resnet50.preprocess_input(image)
    return image, label

# Create training dataset
train_ds_resnet = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds_resnet = train_ds_resnet.shuffle(buffer_size=50000) \
    .map(preprocess_resnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# Create validation (test) dataset
val_ds_resnet = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds_resnet = val_ds_resnet.map(preprocess_resnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)


# ------------------------------------------
# 4. Define Baseline Models to Fine-Tune
# ------------------------------------------
def create_resnet50_finetune(input_shape, num_classes):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze base model
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ----------------------------------------------
# 5. Fine-Tune ResNet50 on CIFAR-100
# ----------------------------------------------
resnet_model = create_resnet50_finetune((224, 224, 3), num_classes)
resnet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning ResNet50 on CIFAR-100 ---")
history_resnet = resnet_model.fit(
    train_ds_resnet,
    epochs=3,
    validation_data=val_ds_resnet,
    verbose=1
)
resnet_loss, resnet_acc = resnet_model.evaluate(val_ds_resnet, verbose=0)
print(f"ResNet50 - CIFAR-100 Accuracy: {resnet_acc:.4f}")



--- Fine-Tuning ResNet50 on CIFAR-100 ---
Epoch 1/3
92/92 [==============================] - 48s 95ms/step - loss: 1.6381 - accuracy: 0.3004 - val_loss: 1.5210 - val_accuracy: 0.3351
Epoch 2/3
92/92 [==============================] - 6s 68ms/step - loss: 1.5180 - accuracy: 0.3532 - val_loss: 1.5176 - val_accuracy: 0.3229
Epoch 3/3
92/92 [==============================] - 7s 73ms/step - loss: 1.5082 - accuracy: 0.3420 - val_loss: 1.4880 - val_accuracy: 0.3665
ResNet50 - CIFAR-100 Accuracy: 0.3665


In [12]:
# ------------------------------
# 3. tf.data Pipeline for EfficientNet
# ------------------------------
def preprocess_efficientnet(image, label):
    # Resize and apply EfficientNet-specific preprocessing
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds_eff = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds_eff = train_ds_eff.shuffle(buffer_size=50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds_eff = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds_eff = val_ds_eff.map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)



def create_efficientnet_finetune(input_shape, num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze base model
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model



# ----------------------------------------------
# 6. Fine-Tune EfficientNetB0 on CIFAR-100
# ----------------------------------------------
eff_model = create_efficientnet_finetune((224, 224, 3), num_classes)
eff_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning EfficientNetB0 on CIFAR-100 ---")
history_eff = eff_model.fit(
    train_ds_eff,
    epochs=3,
    validation_data=val_ds_eff,
    verbose=1
)
eff_loss, eff_acc = eff_model.evaluate(val_ds_eff, verbose=0)
print(f"EfficientNetB0 - CIFAR-100 Accuracy: {eff_acc:.4f}")



--- Fine-Tuning EfficientNetB0 on CIFAR-100 ---
Epoch 1/3
1563/1563 [==============================] - 109s 67ms/step - loss: 0.5143 - accuracy: 0.8315 - val_loss: 0.3506 - val_accuracy: 0.8838
Epoch 2/3
1563/1563 [==============================] - 100s 64ms/step - loss: 0.2873 - accuracy: 0.9016 - val_loss: 0.3390 - val_accuracy: 0.8914
Epoch 3/3
1563/1563 [==============================] - 96s 62ms/step - loss: 0.2076 - accuracy: 0.9289 - val_loss: 0.3440 - val_accuracy: 0.8962
EfficientNetB0 - CIFAR-100 Accuracy: 0.8962


In [ ]:
################
################
#Magnitude-Based Pruning
################
################

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------
# 1. Load TF Flowers Dataset
# ---------------------------------------------
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
IMG_SIZE = 128

# ---------------------------------------------
# 2. Convert Dataset to NumPy Arrays
# ---------------------------------------------
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []

for img, label in dataset_train:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

x_train = np.stack(x_train_list).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(np.array(y_train_list), num_classes)

x_test = np.stack(x_test_list).astype("float32") / 255.0
y_test = tf.keras.utils.to_categorical(np.array(y_test_list), num_classes)

# ---------------------------------------------
# 3. Data Pipeline for EfficientNetB0
# ---------------------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# ---------------------------------------------
# 4. Define Pruned EfficientNetB0 Model
# ---------------------------------------------
def create_pruned_efficientnet(input_shape, num_classes, pruning_params):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model

    pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=pruning_params["initial_sparsity"],
        final_sparsity=pruning_params["final_sparsity"],
        begin_step=pruning_params["begin_step"],
        end_step=pruning_params["end_step"]
    )

    # Apply pruning to the dense layers
    x = layers.Flatten()(base_model.output)
    x = tfmot.sparsity.keras.prune_low_magnitude(
        layers.Dense(256, activation='relu'),
        pruning_schedule=pruning_schedule
    )(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)

    return model

# Define pruning parameters
pruning_params = {
    "initial_sparsity": 0.1,  # Start pruning at 10%
    "final_sparsity": 0.5,    # End pruning at 50%
    "begin_step": 100,        # Start pruning after 100 steps
    "end_step": 3000          # End pruning at step 3000
}

pruned_model = create_pruned_efficientnet((224, 224, 3), num_classes, pruning_params)
pruned_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ---------------------------------------------
# 5. Train the Pruned Model
# ---------------------------------------------
print("\n--- Fine-Tuning Pruned EfficientNetB0 on TF Flowers ---")

history_pruned = pruned_model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 6. Evaluate the Pruned Model
# ---------------------------------------------
pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
print(f"Pruned EfficientNetB0 - TF Flowers Accuracy: {pruned_acc:.4f}")

# ---------------------------------------------
# 7. Strip Pruning for Inference
# ---------------------------------------------
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)
final_model.save("efficientnet_pruned.h5")  # Save the pruned model


In [11]:
################
################
#Random Pruning
################
################

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_model_optimization as tfmot
import random
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------
# 1. Load TF Flowers Dataset
# ---------------------------------------------
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
IMG_SIZE = 128

# ---------------------------------------------
# 2. Convert Dataset to NumPy Arrays
# ---------------------------------------------
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []

for img, label in dataset_train:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

x_train = np.stack(x_train_list).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(np.array(y_train_list), num_classes)

x_test = np.stack(x_test_list).astype("float32") / 255.0
y_test = tf.keras.utils.to_categorical(np.array(y_test_list), num_classes)

# ---------------------------------------------
# 3. Data Pipeline for EfficientNetB0
# ---------------------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# ---------------------------------------------
# 4. Define Random Pruning Function
# ---------------------------------------------
def random_prune_weights(weights, prune_ratio):
    """
    Randomly prunes a given percentage of weights.
    """
    mask = np.ones_like(weights)
    total_weights = weights.size
    num_prune = int(total_weights * prune_ratio)
    
    indices = random.sample(range(total_weights), num_prune)
    np.put(mask, indices, 0)  # Set randomly selected weights to zero
    
    return weights * mask  # Apply mask

# ---------------------------------------------
# 5. Define Randomly Pruned EfficientNetB0 Model
# ---------------------------------------------
def create_random_pruned_efficientnet(input_shape, num_classes, prune_ratio=0.5):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model

    x = layers.Flatten()(base_model.output)
    dense1 = layers.Dense(256, activation='relu')
    x = dense1(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=outputs)
    
    # Apply random pruning to the dense layers
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            weights, biases = layer.get_weights()
            pruned_weights = random_prune_weights(weights, prune_ratio)
            layer.set_weights([pruned_weights, biases])

    return model

# ---------------------------------------------
# 6. Train the Randomly Pruned Model
# ---------------------------------------------
prune_ratio = 0.5  # 50% of the weights are randomly pruned

random_pruned_model = create_random_pruned_efficientnet((224, 224, 3), num_classes, prune_ratio)
random_pruned_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning Randomly Pruned EfficientNetB0 on TF Flowers ---")

history_random_pruned = random_pruned_model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 7. Evaluate the Randomly Pruned Model
# ---------------------------------------------
random_pruned_loss, random_pruned_acc = random_pruned_model.evaluate(val_ds, verbose=0)
print(f"Randomly Pruned EfficientNetB0 - TF Flowers Accuracy: {random_pruned_acc:.4f}")

# Save the final pruned model
random_pruned_model.save("efficientnet_random_pruned.h5")


In [ ]:
################
################
#Gradient-Based Pruning
################
################

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import EfficientNetB0, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------
# 1. Load TF Flowers Dataset
# ---------------------------------------------
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
IMG_SIZE = 128

# ---------------------------------------------
# 2. Convert Dataset to NumPy Arrays
# ---------------------------------------------
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []

for img, label in dataset_train:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

x_train = np.stack(x_train_list).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(np.array(y_train_list), num_classes)

x_test = np.stack(x_test_list).astype("float32") / 255.0
y_test = tf.keras.utils.to_categorical(np.array(y_test_list), num_classes)

# ---------------------------------------------
# 3. Data Pipeline for EfficientNetB0
# ---------------------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# ---------------------------------------------
# 4. Define Function to Compute Weight Importance
# ---------------------------------------------
def compute_gradient_importance(model, train_data, prune_ratio=0.5):
    """
    Computes the gradient-based importance of weights.
    Prunes weights with lowest gradient magnitudes.
    """
    # Get model weights
    layer_weights = [layer.get_weights() for layer in model.layers if isinstance(layer, layers.Dense)]
    
    # Compute gradients
    with tf.GradientTape() as tape:
        loss = sum(model.loss(y, model(x)) for x, y in train_data) / len(train_data)
    grads = tape.gradient(loss, model.trainable_variables)

    # Get absolute values of gradients
    grad_magnitudes = [tf.abs(g) for g in grads if g is not None]

    # Apply pruning
    for (weights, biases), grad in zip(layer_weights, grad_magnitudes):
        threshold = tf.math.top_k(tf.reshape(grad, [-1]), int(prune_ratio * tf.size(grad).numpy())).values[-1]
        mask = tf.cast(grad >= threshold, tf.float32)
        pruned_weights = weights * mask.numpy()  # Set less important weights to zero
        
        # Set new pruned weights
        for layer in model.layers:
            if isinstance(layer, layers.Dense):
                layer.set_weights([pruned_weights, biases])

    return model

# ---------------------------------------------
# 5. Define EfficientNetB0 Model
# ---------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ---------------------------------------------
# 6. Train the Model Before Pruning
# ---------------------------------------------
prune_ratio = 0.5  # 50% of weights will be pruned

# Create model
gradient_pruned_model = create_efficientnet((224, 224, 3), num_classes)
gradient_pruned_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Pretraining EfficientNetB0 Before Pruning ---")

history = gradient_pruned_model.fit(
    train_ds,
    epochs=1,  # Pre-train for 1 epoch to compute gradients
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 7. Apply Gradient-Based Pruning
# ---------------------------------------------
gradient_pruned_model = compute_gradient_importance(gradient_pruned_model, train_ds, prune_ratio)

# ---------------------------------------------
# 8. Train the Pruned Model
# ---------------------------------------------
print("\n--- Fine-Tuning Gradient-Based Pruned EfficientNetB0 ---")

history_pruned = gradient_pruned_model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 9. Evaluate the Pruned Model
# ---------------------------------------------
pruned_loss, pruned_acc = gradient_pruned_model.evaluate(val_ds, verbose=0)
print(f"Gradient-Based Pruned EfficientNetB0 - TF Flowers Accuracy: {pruned_acc:.4f}")

# Save the final pruned model
gradient_pruned_model.save("efficientnet_gradient_pruned.h5")


In [ ]:
################
################
#Variational / Bayesian Pruning
################
################

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_model_optimization as tfmot
import tensorflow_probability as tfp
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0, efficientnet
from tensorflow.keras.optimizers import Adam

tfd = tfp.distributions

# ---------------------------------------------
# 1. Load TF Flowers Dataset
# ---------------------------------------------
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
IMG_SIZE = 128

# ---------------------------------------------
# 2. Convert Dataset to NumPy Arrays
# ---------------------------------------------
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []

for img, label in dataset_train:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

x_train = np.stack(x_train_list).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(np.array(y_train_list), num_classes)

x_test = np.stack(x_test_list).astype("float32") / 255.0
y_test = tf.keras.utils.to_categorical(np.array(y_test_list), num_classes)

# ---------------------------------------------
# 3. Data Pipeline for EfficientNetB0
# ---------------------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# ---------------------------------------------
# 4. Define Bayesian Layers for Pruning
# ---------------------------------------------
def prior(kernel_size, bias_size, dtype=None):
    """
    Defines the prior probability distribution for the weights.
    Uses a standard Gaussian prior (zero mean, unit variance).
    """
    n = kernel_size + bias_size
    return lambda t: tfd.MultivariateNormalDiag(loc=tf.zeros(n), scale_diag=tf.ones(n))

def posterior(kernel_size, bias_size, dtype=None):
    """
    Defines the posterior probability distribution for the weights.
    It is a learnable Gaussian distribution with trainable mean and std.
    """
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(loc=t, scale_diag=tf.nn.softplus(t)))
    ])

# ---------------------------------------------
# 5. Define Bayesian Pruned EfficientNetB0 Model
# ---------------------------------------------
def create_bayesian_efficientnet(input_shape, num_classes, dropout_rate=0.5):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model

    x = layers.Flatten()(base_model.output)

    # Bayesian Dense Layer
    x = tfp.layers.DenseVariational(
        units=256,
        make_prior_fn=prior,
        make_posterior_fn=posterior,
        kl_weight=1 / x_train.shape[0],  # KL divergence weight (scaling factor)
        activation='relu'
    )(x)

    x = layers.Dropout(dropout_rate)(x)  # Bayesian Dropout
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ---------------------------------------------
# 6. Train the Bayesian Model with Pruning
# ---------------------------------------------
bayesian_model = create_bayesian_efficientnet((224, 224, 3), num_classes, dropout_rate=0.5)

bayesian_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning Bayesian Pruned EfficientNetB0 on TF Flowers ---")

history_bayesian = bayesian_model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 7. Evaluate the Pruned Bayesian Model
# ---------------------------------------------
bayesian_loss, bayesian_acc = bayesian_model.evaluate(val_ds, verbose=0)
print(f"Bayesian Pruned EfficientNetB0 - TF Flowers Accuracy: {bayesian_acc:.4f}")

# Save the final pruned model
bayesian_model.save("efficientnet_bayesian_pruned.h5")


In [ ]:
################
################
#Structured Pruning via Group Regularization
################
################

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB0, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------
# 1. Load TF Flowers Dataset
# ---------------------------------------------
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
IMG_SIZE = 128

# ---------------------------------------------
# 2. Convert Dataset to NumPy Arrays
# ---------------------------------------------
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []

for img, label in dataset_train:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

x_train = np.stack(x_train_list).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(np.array(y_train_list), num_classes)

x_test = np.stack(x_test_list).astype("float32") / 255.0
y_test = tf.keras.utils.to_categorical(np.array(y_test_list), num_classes)

# ---------------------------------------------
# 3. Data Pipeline for EfficientNetB0
# ---------------------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# ---------------------------------------------
# 4. Define Structured Pruned EfficientNetB0 Model
# ---------------------------------------------
def create_structured_pruned_efficientnet(input_shape, num_classes, l1_lambda=1e-4, l2_lambda=1e-4):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model

    x = layers.Flatten()(base_model.output)

    # Add Group Lasso Regularization (L1 for sparsity, L2 for structure)
    x = layers.Dense(
        256,
        activation='relu',
        kernel_regularizer=regularizers.l1_l2(l1=l1_lambda, l2=l2_lambda)  # Group regularization
    )(x)

    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ---------------------------------------------
# 5. Train the Structured Pruned Model
# ---------------------------------------------
structured_pruned_model = create_structured_pruned_efficientnet((224, 224, 3), num_classes, l1_lambda=1e-4, l2_lambda=1e-4)

structured_pruned_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning Structured Pruned EfficientNetB0 on TF Flowers ---")

history_structured = structured_pruned_model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
    verbose=1
)

# ---------------------------------------------
# 6. Evaluate the Structured Pruned Model
# ---------------------------------------------
structured_loss, structured_acc = structured_pruned_model.evaluate(val_ds, verbose=0)
print(f"Structured Pruned EfficientNetB0 - TF Flowers Accuracy: {structured_acc:.4f}")

# Save the final pruned model
structured_pruned_model.save("efficientnet_structured_pruned.h5")
